# Test Data Preprocessing

This notebook contains data cleaning and feature engineering for the test data.

In [ ]:
# Import libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from skrub import SimilarityEncoder
from fancyimpute import IterativeImputer

In [ ]:
# Importing the custom functions
import sys
import os 
sys.path.append(os.path.abspath('../src'))
from utils import *